In [1]:
import tensorflow as tf
import json
import numpy as np
import pandas as pd

2025-08-24 10:20:26.486228: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756030826.715406      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756030826.786371      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
data=[]
with open("/kaggle/input/english-hinglish/hinglish_upload_v1.json","r",encoding="utf-8") as f:
    for line in f :
        obj = json.loads(line)
        data.append({
            "English":obj["translation"]["en"],
            "Hinglish":obj["translation"]["hi_ng"]
        })

In [3]:
df = pd.DataFrame(data)
df.head()

,English,Hinglish
0,What's the name of the movie,film ka kya naam hai
1,"Hi, the rotten tomatoes score is great but the...","namaste, sada hua tomatoes score mahaan hai, l..."
2,Do you think you will like the movie,kya aapako lagata hai ki aapako film pasand aa...
3,What kind of movie is it,yah kis tarah kee philm hai
4,when was the movie made?,film kab banee thee?


In [4]:
df.describe()

,English,Hinglish
count,189102,189102
unique,165608,155544
top,cancel all alarms,alarm ko stop kare
freq,64,88


In [5]:
df.isnull().sum()

English     0
Hinglish    0
dtype: int64

In [6]:
df_final = df.sample(frac=1).reset_index(drop=True)

In [7]:
df_final.head()

,English,Hinglish
0,Are there any traffic jams on Route 66 ?,Kya Route 66 par koi traffic jams hai ?
1,Find fastest route,sabse taiz raasta khoje
2,cancel all alarms,sarey alarms cance kare
3,where should we go this weekend,is weekend par humko kahan jaana chahiye
4,Reply to message with voice,message ko voice ke sath reply kare


In [8]:
en_sentence = df_final["English"]
en_length = en_sentence.apply(lambda x:len(str(x).split()))
max_en_length = en_length.max()
max_en_length
     

247

In [9]:
en_sentence

0             Are there any traffic jams on Route 66 ?
1                                   Find fastest route
2                                    cancel all alarms
3                      where should we go this weekend
4                          Reply to message with voice
                              ...                     
189097                            I need gas remind me
189098         How much is it going to rain tomorrow ?
189099    I want to listen to Beyonce ' s latest album
189100               Is the traffic heavy in Flagstaff
189101                                       End timer
Name: English, Length: 189102, dtype: object

In [10]:
hing_sentence = df_final["Hinglish"]
hing_length = hing_sentence.apply(lambda x:len(str(x).split()))
max_hing_length = hing_length.max()
max_hing_length

273

In [11]:
hing_sentence

0             Kya Route 66 par koi traffic jams hai ?
1                             sabse taiz raasta khoje
2                             sarey alarms cance kare
3            is weekend par humko kahan jaana chahiye
4                 message ko voice ke sath reply kare
                             ...                     
189097    mujhe mujhe yaad dilane ke liye gas chahiye
189098               kal kitni barish hone wali hai ?
189099    Mai Beyonce ka latest album sunna chahta hu
189100            Kya Flagstaff me traffic bhaari hai
189101                              timer ko End kare
Name: Hinglish, Length: 189102, dtype: object

In [12]:
vocab_size = 2000
max_length = 100

text_vec_layer_en =tf.keras.layers.TextVectorization(
    vocab_size,output_sequence_length=max_length
) 
text_vec_layer_hing = tf.keras.layers.TextVectorization(
    vocab_size,output_sequence_length=max_length 
    
)
text_vec_layer_en.adapt(en_sentence)
text_vec_layer_hing.adapt([f"startofseq {s} endofseq"for s in hing_sentence])

I0000 00:00:1756030841.902563      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [13]:
vocab_en = text_vec_layer_en.get_vocabulary()
vocab_en = [str(word) for word in vocab_en]
print(vocab_en[:10])
     

['', '[UNK]', 'the', 'to', 'for', 'is', 'i', 'my', 'in', 'me']


In [14]:

vocab_hing = text_vec_layer_hing.get_vocabulary()
vocab_hing = [str(word) for word in vocab_hing]
print(vocab_hing[:10])
     

['', '[UNK]', 'startofseq', 'endofseq', 'ke', 'liye', 'hai', 'ko', 'kya', 'me']


In [15]:
X_seq_enc = text_vec_layer_en(en_sentence)

X_seq_dec = text_vec_layer_hing([f"startofseq {s}" for s in hing_sentence])

Y_seq = text_vec_layer_hing([f"{s} endofseq" for s in hing_sentence])

In [16]:
X_train_enc = X_seq_enc[:150_000]
X_valid_enc = X_seq_enc[150_000:]
X_train_dec = X_seq_dec[:150_000]
X_valid_dec = X_seq_dec[150_000:]
Y_train = Y_seq[:150_000]
Y_valid = Y_seq[150_000:]

In [17]:
a=X_train_enc[10000:10010]

In [18]:
embed_size=128
encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size,embed_size,
                                                    mask_zero=True)
b=encoder_embedding_layer(a)

In [19]:
b

<tf.Tensor: shape=(10, 100, 128), dtype=float32, numpy=
array([[[-0.02991272, -0.03227437,  0.02884299, ..., -0.03203883,
         -0.03040835,  0.03215423],
        [ 0.04628147,  0.04486995,  0.04648042, ..., -0.02241925,
         -0.00357161,  0.00693961],
        [ 0.03329508,  0.01386921,  0.01362223, ...,  0.04035142,
          0.03993157, -0.02667939],
        ...,
        [ 0.0453087 ,  0.00288378, -0.0016626 , ..., -0.03881149,
          0.04866917, -0.01564292],
        [ 0.0453087 ,  0.00288378, -0.0016626 , ..., -0.03881149,
          0.04866917, -0.01564292],
        [ 0.0453087 ,  0.00288378, -0.0016626 , ..., -0.03881149,
          0.04866917, -0.01564292]],

       [[ 0.04626154,  0.03523339,  0.04873784, ..., -0.03598849,
          0.00048033, -0.02848819],
        [-0.03637211, -0.00832649,  0.03239727, ...,  0.0316673 ,
         -0.00624478,  0.02281975],
        [-0.03286581, -0.0122888 , -0.02252342, ..., -0.01756517,
         -0.01146618, -0.04813715],
        ...

In [20]:
Y_train.shape

TensorShape([150000, 100])

In [21]:
Y_train = np.expand_dims(Y_train,-1)
Y_valid = np.expand_dims(Y_valid,-1)

In [22]:
print(Y_train.shape)
print(Y_valid.shape)

(150000, 100, 1)
(39102, 100, 1)


In [23]:
encoder_inputs = tf.keras.layers.Input(shape=(max_length,),dtype=tf.int32)
decoder_inputs = tf.keras.layers.Input(shape=(max_length,),dtype=tf.int32)

embed_size = 128
encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size,embed_size,
                                                    mask_zero=True)
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size,embed_size,
                                                    mask_zero=True)
encoder_embeddings  = encoder_embedding_layer(encoder_inputs)
decoder_embeddings = decoder_embedding_layer(decoder_inputs)
encoder = tf.keras.layers.LSTM(512,return_state=True)
encoder_outputs,*encoder_states = encoder(encoder_embeddings)

decoder = tf.keras.layers.LSTM(512,return_sequences=True)
decoder_outputs = decoder(decoder_embeddings,initial_state=encoder_states)

output_layer = tf.keras.layers.Dense(vocab_size,activation="softmax")
Y_proba = output_layer(decoder_outputs)

In [24]:
model = tf.keras.Model(inputs=[encoder_inputs,decoder_inputs],outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(
    (X_train_enc,X_train_dec),Y_train,
    epochs=10,
    validation_data=((X_valid_enc,X_valid_dec),Y_valid)
)

Epoch 1/10


I0000 00:00:1756030858.578958      95 cuda_dnn.cc:529] Loaded cuDNN version 90300


  59/4688 ━━━━━━━━━━━━━━━━━━━━ 2:20 30ms/step - accuracy: 0.0134 - loss: 6.1106

InvalidArgumentError: Graph execution error:

Detected at node functional_1/lstm_1/Assert/Assert defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 211, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python3.11/